# NSGA-II (Non-dominated Sorting Genetic Algorithm II)

NSGA-2 is apopular multi-objective optimization algorithm. It is widely used for solving problems with multiple conflicting objectives. The algorithm incorporates mechanisms for maintaining diversity in the population while converging toward the Pareto-optimal front.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nsga_2.algo import (
    create_population, objective, assign_fronts, calculate_crowding_distance, 
    tournament_select, sbx_crossover, polynomial_mutation,
    generate_offspring, flatten_fronts
)

## Parameters

In [ ]:
MAX_VALUE = 5
BOUNDS = (-MAX_VALUE, MAX_VALUE)
N_POP = 10
N_GENS = 10

## Objectives

In [ ]:
f1 = lambda x: x ** 2
f2 = lambda x: (x - 2) ** 2

## Core Loop

In [ ]:
p = create_population(N_POP, BOUNDS)

In [ ]:
# project population into objective space and plot results

p_obj = objective(p, (f1, f2))

# plotting
fig, ax = plt.subplots(figsize=(8, 5))
plt.scatter(x=p_obj[:, 0], y=p_obj[:, 1])
ax.set(title="Initial population [objective space]", xlabel="$f_1(x)$", ylabel="$f_2(x)$")

In [ ]:
for gen in range(N_GENS):
    print(f"Generation: {gen}")
    
    p_obj = objective(p, (f1, f2))

    # plotting
    fig, ax = plt.subplots(figsize=(8, 5))
    plt.scatter(x=p_obj[:, 0], y=p_obj[:, 1])
    ax.set(title=f"Generation {gen}", xlabel="$f_1(x)$", ylabel="$f_2(x)$")
    plt.savefig(f"../figures/frames/gen_{gen}.png")
    plt.close()

    fronts = assign_fronts(p_obj)

    crowding_distances = calculate_crowding_distance(p_obj)

    Q = generate_offspring(p_obj, p, fronts, crowding_distances, BOUNDS)

    R = np.vstack((p, Q))
    assert R.shape[0] == 2 * N_POP

    r_obj = objective(R, (f1, f2))
    r_fronts = assign_fronts(r_obj)
    r_crowding_distances = calculate_crowding_distance(r_obj)

    r_combined = np.column_stack((R.reshape(-1), flatten_fronts(r_obj, r_fronts), r_crowding_distances))
    sorted_indices = np.lexsort((r_combined[:, -2], -r_combined[:, -1]))
    r_combined = r_combined[sorted_indices]
    r_combined = r_combined[:N_POP, :]
    p = r_combined[:, :p.shape[1]]
    